In [ ]:
import sys
import re
import numpy as np
from datetime import datetime
import pandas as pd
import json

In [ ]:
#Retweets

userdata = pd.DataFrame(columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count' ))
edges = pd.DataFrame(columns=('Source','Target','Strength'))

with open('combined_data.json', 'r') as f:
    file = json.load(f)
    for tweet in file:
        if 'retweeted_status' not in tweet:
            continue
        userdata = userdata.append(pd.DataFrame([[tweet['user']['id_str'],
                                    tweet['user']['screen_name'],
                                    tweet['user']['created_at'],
                                    tweet['user']['profile_image_url_https'],
                                    tweet['user']['followers_count'],
                                    tweet['user']['friends_count']]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
        userdata = userdata.append(pd.DataFrame([[tweet['retweeted_status']['user']['id_str'],
                                    tweet['retweeted_status']['user']['screen_name'],
                                    tweet['retweeted_status']['user']['created_at'],
                                    tweet['retweeted_status']['user']['profile_image_url_https'],
                                    tweet['retweeted_status']['user']['followers_count'],
                                    tweet['retweeted_status']['user']['friends_count']]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)                 
        edges = edges.append(pd.DataFrame([[tweet['user']['id_str'],
                                    tweet['retweeted_status']['user']['id_str'],
                                    str(datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))]]
                                    , columns=('Source','Target','Strength')), ignore_index=True)


In [ ]:
# Mentions

userdata = pd.DataFrame(columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count' ))
edges = pd.DataFrame(columns=('Source','Target','Strength'))

with open('combined_data.json', 'r') as f:
    file = json.load(f)
    for tweet in file:
        if len(tweet['entities']['user_mentions']) == 0:
            continue

        for mention in tweet['entities']['user_mentions']:
            userdata = userdata.append(pd.DataFrame([[tweet['user']['id_str'],
                                    tweet['user']['screen_name'],
                                    tweet['user']['created_at'],
                                    tweet['user']['profile_image_url_https'],
                                    tweet['user']['followers_count'],
                                    tweet['user']['friends_count']]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
            
            edges = edges.append(pd.DataFrame([[tweet['user']['id_str'],
                                        mention['id_str'],
                                        str(datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))]]
                                        , columns=('Source','Target','Strength')), ignore_index=True)

In [ ]:
# Replies

userdata = pd.DataFrame(columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count' ))
edges = pd.DataFrame(columns=('Source','Target','Strength'))

with open('combined_data.json', 'r') as f:
    file = json.load(f)
    for tweet in file:
        if tweet['in_reply_to_user_id_str'] is None:
            continue

        userdata = userdata.append(pd.DataFrame([[tweet['user']['id_str'],
                                    tweet['user']['screen_name'],
                                    tweet['user']['created_at'],
                                    tweet['user']['profile_image_url_https'],
                                    tweet['user']['followers_count'],
                                    tweet['user']['friends_count']]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
        if len(userdata[userdata['Id'].str.contains(tweet['in_reply_to_user_id_str'])]) == 0:
                userdata = userdata.append(pd.DataFrame([[tweet['in_reply_to_user_id_str'],
                                    tweet['in_reply_to_screen_name'],
                                    np.nan,
                                    np.nan,
                                    np.nan,
                                    np.nan]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
        edges = edges.append(pd.DataFrame([[tweet['user']['id_str'],
                                    tweet['in_reply_to_user_id_str'],
                                    str(datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))]]
                                    , columns=('Source','Target','Strength')), ignore_index=True)

In [ ]:
sl = 20 

edges2 = edges.groupby(['Source','Target'])['Strength'].count()
edges2 = edges2.reset_index()
edges2 = edges2[edges2['Strength'] >= sl]

In [ ]:
userdata = userdata.sort_values(['Id','followers_count'], ascending=[True, False])
userdata = userdata.drop_duplicates(['Id'], keep='first') 

ids = edges2['Source'].append(edges2['Target']).to_frame()
ids.columns = ['Id']
ids = ids.drop_duplicates()

nodes = pd.merge(ids, userdata, on='Id', how='left')

In [ ]:
nodes.to_csv('nodes_@_20.csv', encoding='utf-8', index=False)
edges.to_csv('edges_@_20.csv', encoding='utf-8', index=False)